# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [67]:
sc

Calculation started (calculation_id=28c9595f-c54a-9924-2335-2cf3563d15d7) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [68]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=0cc9595f-c673-3586-b2c9-74cad199c5f6) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [69]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size='large')
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)


Calculation started (calculation_id=d8c9595f-c78c-4d86-02c4-f92dcff757be) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3.1 [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [70]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'hbae35'

Calculation started (calculation_id=ecc9595f-c8a0-981e-bf3e-940e6f126cd3) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.2 [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [71]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    trips = trips.filter(trips.trip_distance >= 2)
    return trips
   # pass

Calculation started (calculation_id=fcc9595f-c9ca-0a70-c516-05161e6c9f3a) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [72]:
#export
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
    mtrips = trips.join(lookup, [trips.DOLocationID == lookup.LocationID\
                        , lookup.Borough == "Manhattan"]\
   ,'inner').select(trips.DOLocationID, trips.passenger_count)
    #mtrips = mtrips.filter(~col("DOLocationID").isNull())
    mtrips = mtrips.groupby("DOLocationID").agg(sum("passenger_count")\
    .alias("pcount")).orderBy("pcount",ascending=False).limit(20)
    return mtrips


Calculation started (calculation_id=ccc9595f-caf3-71bf-59aa-6551f8445008) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [73]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe

    df = trips.join(mtrips, on=['DOLocationID'] , how='left')
    df2 = df.groupBy('PULocationID').agg(mean('total_amount').alias('avg_total_amount'),
                                               count('PULocationID').alias('trip_count'),
                                               count('pcount').alias('top20_count'))
    
    df_final = df2\
        .withColumn('weighted_profit', (col('top20_count')/col('trip_count')) * col('avg_total_amount'))\
        .filter(col('weighted_profit')!=0)\
        .select('PULocationID', col('weighted_profit'))
    
    return df_final

Calculation started (calculation_id=72c9595f-cc10-70fa-ae17-91e2b7a2f53a) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.4 [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [74]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    lookup = lookup.withColumnRenamed('LocationID', 'PULocationID')
    df_final = wp.join(lookup, on= ['PULocationID'], how = 'left')\
            .sort(desc('weighted_profit'))\
            .select('Zone','Borough','weighted_profit')\
            .limit(20)
    
    return df_final

Calculation started (calculation_id=5ac9595f-cd13-0076-0efe-8fe137e22c95) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [75]:
trips, lookup = load_data(size='large')

Calculation started (calculation_id=54c9595f-ce46-001a-4b76-bf9d6b344be0) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
Trip Count:  187203269



In [76]:
mtrips = manhattan_trips(trips, lookup)

Calculation started (calculation_id=5ec95960-2418-b89c-e4d6-f5eddad9b8e9) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [77]:
wp = weighted_profit(trips,mtrips)

Calculation started (calculation_id=fec95960-2911-19a0-ca90-b55080880ddb) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [78]:
final = final_output(wp,lookup)
final.show()
#final.write.csv('s3://cse6242-hbae35/q3-large_final')

Calculation started (calculation_id=cac95960-2e2a-0a15-adcd-b5049245be1b) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+--------------------+---------+------------------+
|                Zone|  Borough|   weighted_profit|
+--------------------+---------+------------------+
|Briarwood/Jamaica...|   Queens|29.154274257219733|
|        Baisley Park|   Queens|27.566277844176806|
|Flushing Meadows-...|   Queens|25.892262842959575|
|       South Jamaica|   Queens|23.788918606459475|
|     Randalls Island|Manhattan|18.524747085180493|
|   LaGuardia Airport|   Queens| 18.36148041173493|
|         JFK Airport|   Queens|17.004295438401886|
|Springfield Garde...|   Queens| 16.32148785749215|
|             Jamaica|   Queens|15.426118009407105|
|              Corona|   Queens|14.893942611936087|
|             Maspeth|   Queens|14.572432507577448|
|         Jamaica Bay|   Queens|13.525482350738125|
|        Astoria Park|   Queens|12.669548117214966|
|      Midtown Center|Manhattan|10.326825461187203|
|       Midtown North|Manhattan|10.279861307951032|
|Times Sq/Theatre ...|Manhattan|10.064789

In [79]:
main(size='large',bucket='s3://cse6242-hbae35/q3-large')

Calculation started (calculation_id=e8c95960-8088-24ed-23b6-bb4ef899d988) in (session=9ec95938-20b8-815d-c6fe-e52bddc132f2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
hbae35
Trip Count:  187203269
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens|29.360455779130838|
|Flushing Meadows-...|       Queens|27.304845733617675|
|       South Jamaica|       Queens|26.294916239873466|
|     Randalls Island|    Manhattan| 24.15098994022753|
|        Astoria Park|       Queens| 21.70641711214752|
|Briarwood/Jamaica...|       Queens|19.945064631789336|
|Springfield Garde...|       Queens|19.468309288781906|
|             Jamaica|       Queens|  19.2839430001379|
|              Corona|       Queens|18.228769248155974|
|   LaGuardia Airport|       Queens|18.181338808373003|
|         Jamaica Bay|       Queens|17.100529446757896|
|             Maspeth|       Queens| 17.00545064007954|
|Eltingville/Annad...|Staten Island| 16.83776475694445|
|         JFK Airport|       Queens|16.777725348249